In [2]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Subset
from urllib.parse import urlparse
from three_datasets import all_dataset
import random
tqdm.pandas()
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print(f"✅ Seed fixed to {seed}")

  label                                                url
0     2                        https://blog.sockpuppet.us/
1     2                  https://blog.apiki.com/seguranca/
2     1  http://autoecole-lauriston.com/a/T0RVd056QXlNe...
3     1  http://chinpay.site/index.html?hgcFSE@E$Z*DFcG...
4     2  http://www.firstfivenebraska.org/blog/article/...


c:\Users\rrpra\Documents\Github\git_testing\email_and_url_classifier_using_nlp_feature_extraction_sem_5_project\three_datasets.py:70: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df4['label'][df4['label'] == 1] = 0
c:\Users\rrpra\Documents\


✅ Dataset 1 (Malicious URLs) cleaned: 522214 samples (Phishing=94111, Benign=428103)

📂 Dataset 1 (Malicious URLs) split → Train: 417732, Valid: 52217, Test: 52217

📂 Dataset 1 (Malicious URLs) split → Train: 188296, Valid: 23537, Test: 23537

📂 Dataset 1 (Malicious URLs) split → Train: 528097, Valid: 66012, Test: 66013

📂 Dataset 1 (Malicious URLs) split → Train: 639999, Valid: 80000, Test: 80000
✅ Seed fixed to 42


In [3]:
from transformers import AutoTokenizer,  T5EncoderModel
import torch
import numpy as np

model_name = "google/canine-s"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5EncoderModel.from_pretrained(model_name)  
model.eval()
def get_byt5_embedding(url):
    # Tokenize the input
    inputs = tokenizer(url, return_tensors="pt", truncation=True, padding=True, max_length=50)
    
    # Get encoder output (not generate)
    with torch.no_grad():
        outputs = model(**inputs)
        # outputs.last_hidden_state: [batch_size, seq_len, hidden_dim]
         
        # Mean pooling across sequence
        emb = outputs.last_hidden_state.mean(dim=1).reshape(-1)
    return emb.cpu().numpy()

# Example
#url = "http://paypal-login-secure-update.com"
#vector = get_byt5_embedding(url)
#print("Embedding shape:", vector.shape)  # e.g. (512,) or (768,) depending on model size


c:\Users\rrpra\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\rrpra\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\rrpra\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framewo

In [4]:
import torch
from transformers import AutoTokenizer, AutoModel

# Load once globally (so it doesn't reload every call)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_NAME = "google/canine-s"  # or "google/canine-c" (larger, slower)

# Load model + tokenizer only once
tokenizer_canine = AutoTokenizer.from_pretrained(MODEL_NAME)
model_canine = AutoModel.from_pretrained(MODEL_NAME).to(DEVICE)
model_canine.eval()

@torch.no_grad()
def get_canine_embedding(url, max_length=50, pooling="mean"):
    """
    Returns a fixed-length CANINE embedding for a single URL string.

    Args:
        url (str): URL text to encode.
        max_length (int): Maximum length of character sequence (truncate/pad to this).
        pooling (str): 'mean' or 'max' for pooling token embeddings into one vector.

    Returns:
        np.ndarray: 1D embedding vector (shape [hidden_dim], e.g. [768]).
    """
    import numpy as np

    # Tokenize
    encoded = tokenizer_canine(
        url,
        truncation=True,
        padding="max_length",
        max_length=max_length,
        return_tensors="pt"
    ).to(DEVICE)

    # Forward pass
    outputs = model_canine(**encoded)
    hidden = outputs.last_hidden_state  # [1, seq_len, hidden_dim]
    mask = encoded["attention_mask"].unsqueeze(-1).type_as(hidden)  # [1, seq_len, 1]

    # Pooling
    if pooling == "mean":
        summed = (hidden * mask).sum(dim=1)
        denom = mask.sum(dim=1).clamp(min=1e-9)
        pooled = summed / denom  # [1, hidden_dim]
    elif pooling == "max":
        hidden = hidden.masked_fill(mask == 0, -1e9)
        pooled = hidden.max(dim=1).values  # [1, hidden_dim]
    else:
        raise ValueError("Pooling must be 'mean' or 'max'")

    return pooled.squeeze(0).cpu().numpy()  # (hidden_dim,)


In [5]:
# ============================================================
# Character Encoding Setup
# ============================================================

# Allowed printable ASCII chars
ascii_chars = [chr(i) for i in range(32, 127)]

# Special control tokens
special_tokens = [
    '<PAD>', '<UNK>',
]

# Build vocab and mapping
vocab = special_tokens + ascii_chars
char2idx = {ch: i for i, ch in enumerate(vocab)}



def encode(text, max_len=100):
    indices = torch.full((max_len,), char2idx["<PAD>"], dtype=torch.long)
    text = text.lower()[:max_len]
    for i, c in enumerate(text):
        indices[i] = char2idx.get(c, char2idx["<UNK>"])

    return indices





encoded_data = {}
frac = 1
gen = all_dataset()



for i, splits in zip([f'dataset_{i}' for i in range(4)], gen):
    encoded_data[i] = {}
    print(f"\n🔠 Encoding structured URLs for {i}...")

    for split_name, df in zip(['train', 'valid', 'test'], splits):
        df = df.sample(frac=frac, random_state=42)
        df["encode"] = df["url"].progress_apply(encode)
        encoded_data[i][split_name] = df
        print(f"  ✅ {split_name}: Encoded {len(df)} URLs")
    next(gen)
    next(gen)
    next(gen)

print("\n✅ All datasets encoded with proper start/end markers and padding!")


🔠 Encoding structured URLs for dataset_0...


100%|██████████| 417732/417732 [01:15<00:00, 5503.29it/s]


  ✅ train: Encoded 417732 URLs


100%|██████████| 52217/52217 [00:09<00:00, 5476.90it/s]


  ✅ valid: Encoded 52217 URLs


100%|██████████| 52217/52217 [00:09<00:00, 5555.24it/s]

  ✅ test: Encoded 52217 URLs

✅ All datasets encoded with proper start/end markers and padding!


In [6]:
from torch.utils.data import TensorDataset, DataLoader
# ============================================================
# Convert to TensorDataset and DataLoader
# ============================================================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 128

dataloader_dict = {}

def make_tensor_dataset(df):
    url_tensor = torch.stack(list(df["encode"]))
    labels_tensor = torch.tensor(df["label"].values, dtype=torch.long)
    return TensorDataset(url_tensor, labels_tensor)

for name, splits in encoded_data.items():
    dataloader_dict[name] = {}
    print(f"\n📦 Creating DataLoaders for {name}...")
    
    train_set = make_tensor_dataset(splits["train"])
    val_set = make_tensor_dataset(splits["valid"])
    test_set = make_tensor_dataset(splits["test"])
    
    dataloader_dict[name]["train_loader"] = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    dataloader_dict[name]["val_loader"] = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    dataloader_dict[name]["test_loader"] = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    
    print(f"✅ DataLoaders ready for {name} (Train/Val/Test)")

print("\n🚀 All DataLoaders are ready in `dataloader_dict`!")

# Example Access:
# dataloader_dict["dataset1"]["train_loader"]



📦 Creating DataLoaders for dataset_0...
✅ DataLoaders ready for dataset_0 (Train/Val/Test)

🚀 All DataLoaders are ready in `dataloader_dict`!


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# =====================================================
# 🔹 TinyByT5 Encoder (Reduced depth for short sequences)
# =====================================================
class TinyByT5Encoder(nn.Module):
    def __init__(self,
                 vocab_size=256,
                 d_model=128,
                 num_layers=2,
                 num_heads=2,
                 dim_ff=256,
                 max_len=100,
                 n_out=128,
                 dropout=0.1):
        super().__init__()

        # 🔹 Byte embedding layer (0–255)
        self.embedding = nn.Embedding(vocab_size, d_model)

        # 🔹 Positional embeddings
        self.pos_embedding = nn.Embedding(max_len, d_model)

        # 🔹 Transformer encoder layers
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=num_heads,
            dim_feedforward=dim_ff,
            dropout=dropout,
            activation="gelu",
            batch_first=True,
            norm_first=True,
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # 🔹 Final normalization
        self.final_norm = nn.LayerNorm(d_model)
        self.projection = nn.Linear(in_features=d_model, out_features=n_out)

    def forward(self, x):
        """
        x: (batch, seq_len) — byte indices [0–255]
        """
        batch_size, seq_len = x.size()
        device = x.device

        #positions = torch.arange(seq_len, device=device).unsqueeze(0).expand(batch_size, -1)
        #x = self.embedding(x) + self.pos_embedding(positions)

        #x = self.encoder(x)
        x = self.embedding(x)
        x = self.projection(self.final_norm(x))

        return x  # (B, L, d_model)

In [59]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==========================
# 🔹 Tiny Transformer Encoder
# ==========================
class TinyURLTransformer(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, num_heads=2, num_layers=2, ff_dim=128, max_len=100, out_dim=128):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_embedding = nn.Parameter(torch.randn(1, max_len, embed_dim))

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            dim_feedforward=ff_dim,
            batch_first=True,
            activation="gelu"
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.projection = nn.Linear(embed_dim, out_dim)

    def forward(self, x):
        # x: (batch, seq_len)
        #x = self.embedding(x) + self.pos_embedding[:, :x.size(1), :]  # (B, L, E)
        #x = self.transformer(x)                                       # (B, L, E)
        return self.embedding(x) #self.projection(x)                                     # (B, L, out_dim)

class URLBinaryCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, maxlen=100):
        super(URLBinaryCNN, self).__init__()
        self.maxlen = maxlen

        self.transformer = TinyByT5Encoder(
            vocab_size=vocab_size,
            max_len=maxlen,
            d_model=512,
            n_out=embed_dim
        )
        # 🔹 1st Conv block
        self.conv1_3x3 = nn.Conv1d(in_channels=embed_dim, out_channels=128, kernel_size=3, padding=1)
        self.convreduce1_1x1 = nn.Conv1d(in_channels=embed_dim, out_channels=1, kernel_size=1, padding=0)
        self.conv1_5x5 = nn.Conv1d(in_channels=embed_dim, out_channels=128, kernel_size=5, padding=2)
        self.conv1_7 = nn.Conv1d(in_channels=embed_dim, out_channels=128, kernel_size=7, padding=3)
        self.conv1_1x1 = nn.Conv1d(in_channels=128*3, out_channels=128, kernel_size=1, padding=0)
        self.layer_norm1 = nn.LayerNorm(normalized_shape=[128, maxlen])


         # 🔹 2st Conv block
        self.conv2_3x3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.convreduce2_1x1 = nn.Conv1d(in_channels=128, out_channels=1, kernel_size=3, padding=1)
        self.conv2_5x5 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=5, padding=2)
        self.conv2_1x1 = nn.Conv1d(in_channels=64*2, out_channels=64, kernel_size=3, padding=1)
        self.layer_norm2 = nn.LayerNorm(normalized_shape=[64, maxlen])
        

         # 🔹 3st Conv block
        self.conv3_3x3 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, padding=1)
        self.convreduce3_1x1 = nn.Conv1d(in_channels=64, out_channels=1, kernel_size=1, padding=0)
        self.conv3_5x5 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=5, padding=2)
        self.conv3_7 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=7, padding=3)
        self.conv3_1x1 = nn.Conv1d(in_channels=32*3, out_channels=32, kernel_size=1, padding=0)
        self.layer_norm3 = nn.LayerNorm(normalized_shape=[32, maxlen])

        self.lstm = nn.LSTM(input_size=128, hidden_size=128, num_layers=1, 
                            batch_first=True)
        
        self.aap = nn.AdaptiveAvgPool1d(1)

        # 🔹 Fully connected layers

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc_layers = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.5),



            nn.Linear(64, 1)
        )
    def forward(self, x):
        # (batch, seq_len)
        x = self.transformer(x)           # (B, L, E)
        
        x = x.permute(0, 2, 1)            # (B, E, L)



        # 🔹 Block 1
        x3 = F.relu(self.conv1_3x3(x))

        #xr = F.relu(self.convreduce1_1x1(x))
        x5 = F.relu(self.conv1_5x5(x))
        x7 = F.relu(self.conv1_7(x))

        x = torch.cat([x3, x5, x7], dim=1)
        #print(x.shape)
        x = F.relu(self.conv1_1x1(x))

        #x = self.layer_norm1(x)


        '''
        # 🔹 Block 2
        x3 = F.relu(self.conv2_3x3(x))
        xr = F.relu(self.convreduce2_1x1(x))
        x5 = F.relu(self.conv2_5x5(xr))
        x = torch.cat([x3, x5], dim=1)
        x = F.relu(self.conv2_1x1(x))
        x = self.layer_norm2(x)
        '''
        # 🔹 Block 3
        #x = F.relu(self.conv3_3x3(x))
        #xr = F.relu(self.convreduce3_1x1(x))
        #x5 = F.relu(self.conv3_5x5(x))
        #x7 = F.relu(self.conv3_7(x))
        #x = torch.cat([x3, x5, x7], dim=1)
        #x = F.relu(self.conv3_1x1(x))
        #x = self.layer_norm3(x)

        x = x.permute(0, 2, 1)

        x, _ = self.lstm(x)
        x = x[:, -1, :]
        # 🔹 Global Average Pooling + FC
        #x = self.aap(x)                   # (batch, channels, 1)
        x = self.flatten(x)               # (batch, channels)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc_layers(x)

        return torch.sigmoid(x)
    
    def extract_features(self, x):
        """Return deep features before final FC layers."""
        x = self.transformer(x)
        x = x.permute(0, 2, 1)
        x3 = F.relu(self.conv1_3x3(x))
        x5 = F.relu(self.conv1_5x5(x))
        x7 = F.relu(self.conv1_7(x))
        x = torch.cat([x3, x5, x7], dim=1)
        x = F.relu(self.conv1_1x1(x))
        x = x.permute(0, 2, 1)
        x, _ = self.lstm(x)
        x = x[:, -1, :]          # shape: [batch, 32]
        x = self.flatten(x) 
        x = self.dropout(self.relu(self.fc1(x)))
        return x



In [54]:
import torch
import torch.nn as nn
import torch.optim as optim
import sys

class Train:
    def __init__(self, 
                 model, 
                 criterion, 
                 transformer_optimizer=None,
                 main_optimizer = None, 
                 scheduler_t=None,
                 scheduler_c=None,
                 train_loader=None, 
                 val_loader=None,   
                 device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
        
        """
        optimizer_groups: dict with keys like {"transformer": optimizer1, "cnn": optimizer2}
        schedulers: dict with keys matching optimizer_groups (optional)
        """
        self.model = model
        self.criterion = criterion

        self.transformer_params = []
        self.cnn_params = []
        for name, param in model.named_parameters():
            if name.startswith("transformer."):
                self.transformer_params.append(param)
            else:
                self.cnn_params.append(param)



        self.transformer_optimizer = optim.Adam(self.transformer_params, lr=1e-4) if transformer_optimizer is None  else transformer_optimizer
        self.main_optimizer = optim.Adam(self.cnn_params, lr=1e-3) if main_optimizer is None else main_optimizer
        self.scheduler_t = optim.lr_scheduler.ReduceLROnPlateau(self.transformer_optimizer, mode='min', factor=0.5, patience=2) if scheduler_t is None  else scheduler_t
        self.scheduler_c = optim.lr_scheduler.ReduceLROnPlateau(self.main_optimizer, mode='min', factor=0.5, patience=2) if scheduler_c is None  else scheduler_c
        self.device = device
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.train_losses, self.val_losses = [], []
        self.train_accs, self.val_accs = [], []


    def freeze_module(self, module):
        for param in module.parameters():
            param.requires_grad = False

    def unfreeze_module(self, module):
        for param in module.parameters():
            param.requires_grad = True


    def train(self, epochs_list=[3,3,4], early_stopping=True, frac=1.0, val_frac=1.0, alt_cycle = 2,  log=False):
        for phase, epochs in enumerate(epochs_list):
            for epoch in range(epochs):
                phase=3
                if phase == 0:
                    # 🧠 Train Transformer — freeze CNN/LSTM layers
                    self.unfreeze_module(self.model.transformer)
                    for name, module in self.model.named_children():
                        if name != "transformer":
                            self.freeze_module(module)
                        else:
                            self.unfreeze_module(module)
                    active_optims = [self.transformer_optimizer]
                    active_scheds = [self.scheduler_t]
                    phase_name = "Transformer"
                elif phase == 1:
                    # 🎯 Train CNN/LSTM/FC — freeze Transformer
                    self.freeze_module(self.model.transformer)
                    for name, module in self.model.named_children():
                        if name == "transformer":
                            self.freeze_module(module)
                        else:
                            self.unfreeze_module(module)
                    active_optims = [self.main_optimizer]
                    active_scheds = [self.scheduler_c]
                    phase_name = "CNN"
                else:
                    self.freeze_module(self.model.transformer)
                    for name, module in self.model.named_children():
                        self.unfreeze_module(module)
                    active_optims = [self.transformer_optimizer, self.main_optimizer]
                    active_scheds = [self.scheduler_t, self.scheduler_c]
                    phase_name = "CNN + transformer"



                self.model.train()
                train_loss, correct_train, total_train = 0, 0, 0
                max_batches = int(len(self.train_loader) * frac)
                
                for batch_idx, (batch_x, batch_y) in enumerate(self.train_loader):
                    if batch_idx >= max_batches:
                        break
                    
                    batch_x, batch_y = batch_x.to(self.device, non_blocking=True), batch_y.to(self.device, non_blocking=True).float().unsqueeze(1)
                    
                    for opt in active_optims:
                        opt.zero_grad()

                    outputs = self.model(batch_x)
                    loss = self.criterion(outputs, batch_y)
                    loss.backward()

                    for opt in active_optims:
                        opt.step()




                    # === Metrics ===
                    batch_loss = loss.item()
                    preds = (outputs >= 0.5).float()
                    batch_acc = (preds == batch_y).float().mean().item()

                    train_loss += batch_loss * batch_x.size(0)
                    correct_train += (preds == batch_y).sum().item()
                    total_train += batch_x.size(0)

                    if log and (batch_idx + 1) % 20 == 0:
                        print(f"\rEpoch {epoch+1}/{epochs}: Training {phase_name} | "
                            f"Batch {batch_idx+1}/{max_batches} | "
                            f"Loss: {batch_loss:.4f}, Acc: {batch_acc:.4f}", end='')

                avg_train_loss = train_loss / total_train
                train_acc = correct_train / total_train
                self.train_losses.append(avg_train_loss)
                self.train_accs.append(train_acc)

                # === Validation ===
                if self.val_loader is not None:
                    avg_val_loss, val_acc = self.evaluate(val_frac)
                    self.val_losses.append(avg_val_loss)
                    self.val_accs.append(val_acc)

                    if log:
                        print(f"\rEpoch {epoch+1}/{epochs} Training {phase_name}| "
                            f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f} | "
                            f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}")
                elif log:
                    print(f"\rEpoch {epoch+1}/{epochs} Training {phase_name}| "
                        f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f}")


                for sched in active_scheds:
                    sched.step(avg_val_loss)



    def evaluate(self, frac=1.0):
        self.model.eval()
        val_loss, correct_val, total_val = 0, 0, 0
        max_batches = max(int(len(self.val_loader) * frac), 1)
        print(f'\r total validation batch size {max_batches}'.ljust(100), end='')
        with torch.no_grad():
            for batch_idx, (batch_x, batch_y) in enumerate(self.val_loader):
                if batch_idx >= max_batches:
                    break
                batch_x, batch_y = batch_x.to(self.device, non_blocking=True), batch_y.to(self.device, non_blocking=True).float().unsqueeze(1)
                outputs = self.model(batch_x)
                loss = self.criterion(outputs, batch_y)
                avg_batch_loss = loss.item()
                val_loss += avg_batch_loss * batch_x.size(0)
                preds = (outputs >= 0.5).float()
                correct_val += (preds == batch_y).sum().item()
                total_val += batch_x.size(0)

        avg_val_loss = val_loss / total_val
        val_acc = correct_val / total_val
        return avg_val_loss, val_acc
    



In [60]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from lion_pytorch import Lion
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ============================================================
# 🔧 Training Config
# ============================================================
num_epochs = [3,3,4]
lr = 0.001

# Store all dataset metrics
all_results = {}

# ============================================================
# 🔁 Training Loop for Each Dataset
# ============================================================
for dataset_name, loaders in {'dataset_0':dataloader_dict['dataset_0']}.items():
    print("\n" + "="*70)
    print(f"🚀 Training model on {dataset_name.upper()} dataset")
    print("="*70)
    for frac in  [1.0]:
        print(f"🧩 Using {frac*100:.0f}% of training data".center(50, '_'))
        train_loader = loaders["train_loader"]
        val_loader = loaders["val_loader"]

        # Initialize model, loss, optimizer
        model = URLBinaryCNN(vocab_size=len(vocab)).to(device)
        criterion = nn.BCELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=lr/10)
        trainer = Train(model, criterion, train_loader=train_loader, val_loader=val_loader)
        # Lists to track performance
        trainer.train(num_epochs,lr,frac=frac,val_frac=frac, log=True)

    

    # Store all results for this dataset
    all_results[dataset_name] = {
        "train_losses": trainer.train_losses,
        "val_losses": trainer.val_losses,
        "train_accs": trainer.train_accs,
        "val_accs": trainer.val_accs,
        "final_val_acc": trainer.val_accs[-1],
        "final_val_loss": trainer.val_losses[-1]
    }

print("\n✅ All datasets trained successfully!")

# ============================================================
# 📊 Summary of All Results
# ============================================================
print("\n" + "="*70)
print("📈 Final Validation Accuracy Summary")
print("="*70)
for name, res in all_results.items():
    print(f"{name:<20} | Val Acc: {res['final_val_acc']:.4f} | Val Loss: {res['final_val_loss']:.4f}")



🚀 Training model on DATASET_0 dataset
__________🧩 Using 100% of training data___________
Epoch 1/3 Training CNN + transformer| Train Loss: 0.1497, Train Acc: 0.9480 | Val Loss: 0.0757, Val Acc: 0.9779
Epoch 2/3 Training CNN + transformer| Train Loss: 0.0657, Train Acc: 0.9795 | Val Loss: 0.0545, Val Acc: 0.9816
Epoch 3/3 Training CNN + transformer| Train Loss: 0.0547, Train Acc: 0.9825 | Val Loss: 0.0547, Val Acc: 0.9817
Epoch 1/3 Training CNN + transformer| Train Loss: 0.0486, Train Acc: 0.9843 | Val Loss: 0.0551, Val Acc: 0.9815
Epoch 2/3 Training CNN + transformer| Train Loss: 0.0443, Train Acc: 0.9855 | Val Loss: 0.0512, Val Acc: 0.9824
Epoch 3/3 Training CNN + transformer| Train Loss: 0.0418, Train Acc: 0.9863 | Val Loss: 0.0481, Val Acc: 0.9853
Epoch 1/4 Training CNN + transformer| Train Loss: 0.0398, Train Acc: 0.9870 | Val Loss: 0.0468, Val Acc: 0.9850
Epoch 2/4 Training CNN + transformer| Train Loss: 0.0378, Train Acc: 0.9873 | Val Loss: 0.0474, Val Acc: 0.9858
Epoch 3/4 Trai

In [61]:
torch.save(model.state_dict(), "url_cnn_lstm.pth")


In [14]:
#xgbost model + cnn based dl model
def extract_features(self, x):
        """Return deep features before final FC layers."""
        x = self.transformer(x)
        x = x.permute(0, 2, 1)
        x3 = F.relu(self.conv1_3x3(x))
        x5 = F.relu(self.conv1_5x5(x))
        x7 = F.relu(self.conv1_7(x))
        x = torch.cat([x3, x5, x7], dim=1)
        x = F.relu(self.conv1_1x1(x))
        x = x.permute(0, 2, 1)
        x, _ = self.lstm(x)
        x = x[:, -1, :]          # shape: [batch, 32]
        return x

In [62]:
import numpy as np
from tqdm import tqdm
import torch

model.eval()
features, labels = [], []

# 🔹 Extract CNN features for training set
with torch.no_grad():
    for x_batch, y_batch in tqdm(train_loader, desc="🔍 Extracting Train Features", unit="batch"):
        x_batch = x_batch.to(device, non_blocking=True)
        feats = model.extract_features(x_batch)
        features.append(feats.cpu().numpy())
        labels.append(y_batch.cpu().numpy())

X_train = np.concatenate(features, axis=0)
y_train = np.concatenate(labels, axis=0)

# Free memory before val extraction
del features, labels, x_batch, y_batch, feats
torch.cuda.empty_cache()

# 🔹 Extract CNN features for validation set
features, labels = [], []
with torch.no_grad():
    for x_batch, y_batch in tqdm(val_loader, desc="🔍 Extracting Val Features", unit="batch"):
        x_batch = x_batch.to(device, non_blocking=True)
        feats = model.extract_features(x_batch)
        features.append(feats.cpu().numpy())
        labels.append(y_batch.cpu().numpy())

X_val = np.concatenate(features, axis=0)
y_val = np.concatenate(labels, axis=0)


🔍 Extracting Val Features: 100%|██████████| 408/408 [00:06<00:00, 61.01batch/s] 


In [64]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="logloss"
)

xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=True
)


[0]	validation_0-logloss:0.42642
[1]	validation_0-logloss:0.39052
[2]	validation_0-logloss:0.36064
[3]	validation_0-logloss:0.33507
[4]	validation_0-logloss:0.31273
[5]	validation_0-logloss:0.29294
[6]	validation_0-logloss:0.27524
[7]	validation_0-logloss:0.25927
[8]	validation_0-logloss:0.24476
[9]	validation_0-logloss:0.23152
[10]	validation_0-logloss:0.21936
[11]	validation_0-logloss:0.20820
[12]	validation_0-logloss:0.19788
[13]	validation_0-logloss:0.18834
[14]	validation_0-logloss:0.17947
[15]	validation_0-logloss:0.17123
[16]	validation_0-logloss:0.16358
[17]	validation_0-logloss:0.15645
[18]	validation_0-logloss:0.14980
[19]	validation_0-logloss:0.14356
[20]	validation_0-logloss:0.13774
[21]	validation_0-logloss:0.13229
[22]	validation_0-logloss:0.12717
[23]	validation_0-logloss:0.12237
[24]	validation_0-logloss:0.11787
[25]	validation_0-logloss:0.11363
[26]	validation_0-logloss:0.10966
[27]	validation_0-logloss:0.10592
[28]	validation_0-logloss:0.10240
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

y_pred_prob = xgb_model.predict(X_val)
y_pred = (y_pred_prob > 0.5).astype(int)

print("Accuracy:", accuracy_score(y_val, y_pred))
print("Precision:", precision_score(y_val, y_pred))
print("Recall:", recall_score(y_val, y_pred))
print("F1:", f1_score(y_val, y_pred))
print("ROC AUC:", roc_auc_score(y_val, y_pred_prob))


TypeError: isinstance() arg 2 must be a type, a tuple of types, or a union

In [ ]:
#random forest
from sklearn
model = RandomForestClassifier(
    n_estimators=100,   # number of trees
    max_depth=30,     # tree depth (None = auto)
    random_state=42
)



model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=True
)

In [27]:
#adding xgboost with cnn with handcrafted features
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import numpy as np
import xgboost as xgb
import pandas as pd




In [28]:
import pandas as pd
import re
import math
from urllib.parse import urlparse
import tldextract
from rapidfuzz import process, fuzz



import pickle
f= open('tld_encoding_serise.bin','rb')
tld_stats = pickle.load(file=f)
f.close()
print(type(tld_stats))

import kagglehub
import os
# Download latest version
folder_path = kagglehub.dataset_download("cheedcheed/top1m")

print("Path to dataset files:", folder_path)
csv_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.csv')]

if not csv_files:
    raise FileNotFoundError("No CSV files found in the folder!")

# read the first CSV file
file_path = os.path.join(folder_path, csv_files[0])
alexa_top_1m_domain = pd.read_csv(file_path,header=None,names=['rank', 'domain'])
alexa_domains_set = set(alexa_top_1m_domain['domain'].apply(str.lower))

# --- Helper function: Shannon entropy ---
def safe_parse(url: str):
    """Safely parse URLs, adding http:// if missing and handling bad IPv6 parts."""
    if not isinstance(url, str) or not url.strip():
        return urlparse("http://")  

    # Ensure scheme exists
    if not re.match(r'^[a-zA-Z]+://', url):
        url = 'http://' + url

    # Clean invalid brackets that trigger IPv6 errors
    url = re.sub(r'\[.*?\]', '', url)

    try:
        return urlparse(url)
    except ValueError:
        # fallback: strip more aggressively if still malformed
        url = re.sub(r'[^a-zA-Z0-9:/._\-?&=]', '', url)
        return urlparse(url)
def calculate_entropy(string):
    """Measures randomness of characters in the URL."""
    if not string:
        return 0
    freq = {char: string.count(char) for char in set(string)}
    entropy = -sum((count / len(string)) * math.log2(count / len(string)) for count in freq.values())
    return entropy

# --- Main feature extraction function ---
def extract_handcrafted_features(url):
    features = {}
    if not re.match(r'^[hH]+[tT]+[tT]+[pP]+[sS]+://', url):
        url = 'http://' + url
    parsed = safe_parse(url)
    
    # 1️⃣ Basic structural features
    features['url_length'] = len(url)
    features['hostname_length'] = len(parsed.netloc)
    features['path_length'] = len(parsed.path)
    features['num_dots'] = url.count('.')
    features['num_hyphens'] = url.count('-')
    features['num_digits'] = sum(c.isdigit() for c in url)
    features['num_letters'] = sum(c.isalpha() for c in url)
    features['num_params'] = url.count('?')
    features['num_equals'] = url.count('=')
    features['num_slashes'] = url.count('/')
    features['num_at'] = url.count('@')

    # 2️⃣ Lexical / composition cues
    features['has_https'] = 1 if url.lower().startswith('https') else 0
    features['has_ip'] = 1 if re.search(r'(\d{1,3}\.){3}\d{1,3}', parsed.netloc) else 0
    features['has_subdomain'] = 1 if parsed.netloc.count('.') > 1 else 0
    features['has_suspicious_words'] = 1 if re.search(r'(login|secure|verify|update|free|bank|click)', url.lower()) else 0

    # 3️⃣ Domain / TLD features
    extracted = tldextract.extract(url)
    main_domain = f"{extracted.domain}.{extracted.suffix}"
    if ':' in main_domain:  # remove port
        main_domain = main_domain.split(':')[0]
    features['domain_length'] = len(main_domain)
    features['in_alexa_top1m'] = 1 if main_domain in alexa_domains_set else 0
    '''
    if features['in_alexa_top1m'] == 0 and main_domain:  # only check if domain not in top1M
        # find closest match in Alexa domains
        best_match, score, _ = process.extractOne(main_domain, alexa_domains_set, scorer=fuzz.ratio)
        features['closest_alexa_domain'] = best_match
        features['closest_alexa_score'] = score  # 0-100
    else:
        features['closest_alexa_score'] = 1000  # high score to show that it is original url
    '''
    ext = tldextract.extract(url)
    tld = ext.suffix    # "com", "co.uk", "org"
    features['tld'] = tld if tld else 'unknown'
    features['tld_phish_ratio'] = tld_stats['phish_ratio'][features['tld']]  if tld_stats['phish_ratio'][features['tld']]  else 0.5
    features['tld_total_frequency'] = tld_stats['total'][features['tld']] if tld_stats['total'][features['tld']] else 1

    # 4️⃣ Ratios
    features['digit_ratio'] = features['num_digits'] / (features['url_length'] + 1e-5)
    features['special_char_ratio'] = (features['num_hyphens'] + features['num_dots'] + features['num_slashes']) / (features['url_length'] + 1e-5)

    # 5️⃣ Entropy (measures randomness / obfuscation)
    features['url_entropy'] = calculate_entropy(url)

    # 6️⃣ Misplacement indicators
    # '@' symbol used to hide real domain (like "http://evil.com@legit.com")
    features['at_in_domain'] = 1 if '@' in parsed.netloc else 0
    
    # Double slashes '//' appearing after path (used to trick users)
    features['double_slash_in_path'] = 1 if re.search(r'/.+//', parsed.path) else 0

    return features

<class 'pandas.core.frame.DataFrame'>
Path to dataset files: C:\Users\rrpra\.cache\kagglehub\datasets\cheedcheed\top1m\versions\1


In [33]:
manual_feature_dict = {}

for name, splits in encoded_data.items():
    manual_feature_dict[name] = {}
    print(f"\n📦 Creating DataLoaders for {name}...")
    print(splits)


📦 Creating DataLoaders for dataset_0...
{'train':                                                       url  label  \
230159  nps.gov/badl/parknews/sylvatic-plague-detected...      0   
377597  finance.yahoo.com/news/Medline-Selects-GT-Nexu...      0   
157325                       en.wikipedia.org/wiki/Blumer      0   
318564           manfridayfilms.co.uk/s-Steve_Robinson_CV      0   
159250  dukebasketballreport.com/forums/showthread.php...      0   
...                                                   ...    ...   
132127                  shopping.yahoo.com/gamepad--shop/      0   
304497                      mog.com/artists/mn6489/common      0   
241619                           mobile.disney.go.com/wap      0   
211667                                           gmam.ca/      0   
302464  amazon.com/Forest-Pipes-Disney-Concert-Organ/d...      0   

                                                   encode  
230159  [tensor(80), tensor(82), tensor(85), tensor(16...  
377597  [ten

In [ ]:
from tqdm import tqdm
import pandas as pd

manual_feature_dict = {}

for name, splits in encoded_data.items():
    manual_feature_dict[name] = {}
    print(f"\n📦 Extracting handcrafted features for {name}...")

    # --- Train ---
    print("🔍 Extracting Train Features...")
    train_df = splits["train"]
    manual_feature_dict[name]["train"] = 0
    print(type(manual_feature_dict[name]["train"]))
    manual_feature_dict[name]["train"] = pd.DataFrame({})
    
    manual_feature_dict[name]["train"] = manual_feature_dict[name]["train"].assign(**train_df["url"].progress_apply(lambda url : extract_handcrafted_features(url)))
    manual_feature_dict[name]["train"]["label"] = train_df["label"]

    # --- Test ---
    print("🔍 Extracting Test Features...")
    test_df = splits["test"]
    test_feats = test_df["url"].progress_apply(extract_handcrafted_features)
    manual_feature_dict[name]["test"] = manual_feature_dict[name]["test"].assign(**test_df["url"].progress_apply(lambda url : extract_handcrafted_features(url)))
    manual_feature_dict[name]["test"]["label"] = test_df["label"]

    # ✅ Print shapes safely
    train_shape = manual_feature_dict[name]["train"].shape
    test_shape = manual_feature_dict[name]["test"].shape
    print(f"✅ Done: {name} | Train: {train_shape}, Test: {test_shape}")

print("\n🏁 All manual features extracted successfully with progress bars!")



📦 Extracting handcrafted features for dataset_0...
🔍 Extracting Train Features...
<class 'int'>


TypeError: 'Series' object is not callable

In [ ]:
# ============================================================
# 🚀 XGBoost Training on Multiple Datasets
# ============================================================

import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tqdm import tqdm

# Define handcrafted features
independent_features = [
    'url_length', 'hostname_length', 'path_length',
    'num_dots', 'num_hyphens', 'num_digits', 'num_letters', 'num_params',
    'num_equals', 'num_slashes', 'num_at', 'has_https', 'has_ip',
    'has_subdomain', 'has_suspicious_words', 'domain_length',
    'in_alexa_top1m', 'tld_phish_ratio', 'tld_total_frequency',
    'digit_ratio', 'special_char_ratio', 'url_entropy', 
    'at_in_domain', 'double_slash_in_path'
]
dependent_feature = 'label'

# Dictionary to store results
xgb_results = {}

# Loop through each dataset
for name, splits in manual_feature_dict.items():
       print(f"\n📊 Training XGBoost on {name} ...")

       # Prepare train/test DataFrames
       train_df = pd.DataFrame(splits['train'])
       test_df = pd.DataFrame(splits['test'])
       print(train_df.columns)
       # Ensure all required columns exist
       missing_train = [col for col in independent_features if col not in train_df.columns]
       missing_test = [col for col in independent_features if col not in test_df.columns]
       if missing_train or missing_test:
              print(f"⚠️ Missing columns in {name}:", missing_train + missing_test)
              continue

       # Extract X and y
       X_train = train_df[independent_features]
       y_train = train_df[dependent_feature]

       X_test = test_df[independent_features]
       y_test = test_df[dependent_feature]

       # Convert to numpy (optional, for XGBoost performance)
       X_train = X_train.values
       X_test = X_test.values
       y_train = y_train.values
       y_test = y_test.values

       # Train XGBoost model
       model = XGBClassifier(
              n_estimators=300,
              learning_rate=0.1,
              max_depth=6,
              subsample=0.8,
              colsample_bytree=0.8,
              random_state=42,
              eval_metric='logloss',
              n_jobs=-1
       )

       # Use tqdm progress bar for large datasets
       with tqdm(total=1, desc=f"🚀 Training {name}") as pbar:
              model.fit(X_train, y_train)
              pbar.update(1)

       # Predictions
       y_pred = model.predict(X_test)
       y_prob = model.predict_proba(X_test)[:, 1]

       # Evaluation metrics
       acc = accuracy_score(y_test, y_pred)
       prec = precision_score(y_test, y_pred, zero_division=0)
       rec = recall_score(y_test, y_pred, zero_division=0)
       f1 = f1_score(y_test, y_pred, zero_division=0)
       roc = roc_auc_score(y_test, y_prob)

       # Store results
       xgb_results[name] = {
              "Accuracy": acc,
              "Precision": prec,
              "Recall": rec,
              "F1": f1,
              "ROC AUC": roc
       }

       print(f"\n✅ Results for {name}:")
       print(f"Accuracy:  {acc:.4f}")
       print(f"Precision: {prec:.4f}")
       print(f"Recall:    {rec:.4f}")
       print(f"F1-score:  {f1:.4f}")
       print(f"ROC-AUC:   {roc:.4f}")

print("\n🏁 All datasets completed!")



📊 Training XGBoost on dataset_0 ...
RangeIndex(start=0, stop=417732, step=1)
⚠️ Missing columns in dataset_0: ['url_length', 'hostname_length', 'path_length', 'num_dots', 'num_hyphens', 'num_digits', 'num_letters', 'num_params', 'num_equals', 'num_slashes', 'num_at', 'has_https', 'has_ip', 'has_subdomain', 'has_suspicious_words', 'domain_length', 'in_alexa_top1m', 'tld_phish_ratio', 'tld_total_frequency', 'digit_ratio', 'special_char_ratio', 'url_entropy', 'at_in_domain', 'double_slash_in_path', 'url_length', 'hostname_length', 'path_length', 'num_dots', 'num_hyphens', 'num_digits', 'num_letters', 'num_params', 'num_equals', 'num_slashes', 'num_at', 'has_https', 'has_ip', 'has_subdomain', 'has_suspicious_words', 'domain_length', 'in_alexa_top1m', 'tld_phish_ratio', 'tld_total_frequency', 'digit_ratio', 'special_char_ratio', 'url_entropy', 'at_in_domain', 'double_slash_in_path']

🏁 All datasets completed!


In [ ]:
# Create XGBoost classifier
model = xgb.XGBClassifier(
    n_estimators=100,      # number of boosting rounds
    learning_rate=0.01,     # step size shrinkage
    max_depth=30,           # tree depth
    eval_metric='logloss', # evaluation metric
    use_label_encoder=False
)

# Train the model
model.fit(X_train, y_train)
# Predict
y_pred = model.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


In [ ]:
#random forest
model = RandomForestClassifier(
    n_estimators=100,   # number of trees
    max_depth=30,     # tree depth (None = auto)
    random_state=42
)

# Train model
model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)

# Evaluate model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


In [11]:

# ============================================================
# 🔁 Training Loop for Each Dataset
# ============================================================
for dataset_name, loaders in {'dataset_0':dataloader_dict['dataset_0']}.items():
    print("\n" + "="*70)
    print(f"🚀 Training model on {dataset_name.upper()} dataset")
    print("="*70)
    for frac in  [0.5, 1.0]:
        print(f"🧩 Using {frac*100:.0f}% of training data".center(50, '_'))
        train_loader = loaders["train_loader"]
        val_loader = loaders["val_loader"]

        # Initialize model, loss, optimizer
        model = URLBinaryCNN(vocab_size=len(vocab)).to(device)
        criterion = nn.BCELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=lr/10)
        trainer = Train(model, criterion, optimizer, train_loader, val_loader)
        # Lists to track performance
        trainer.train(num_epochs,lr,frac=frac,val_frac=frac, log=True)

    

    # Store all results for this dataset
    all_results[dataset_name] = {
        "train_losses": trainer.train_losses,
        "val_losses": trainer.val_losses,
        "train_accs": trainer.train_accs,
        "val_accs": trainer.val_accs,
        "final_val_acc": trainer.val_accs[-1],
        "final_val_loss": trainer.val_losses[-1]
    }

print("\n✅ All datasets trained successfully!")

# ============================================================
# 📊 Summary of All Results
# ============================================================
print("\n" + "="*70)
print("📈 Final Validation Accuracy Summary")
print("="*70)
for name, res in all_results.items():
    print(f"{name:<20} | Val Acc: {res['final_val_acc']:.4f} | Val Loss: {res['final_val_loss']:.4f}")



🚀 Training model on DATASET_0 dataset
___________🧩 Using 50% of training data___________


TypeError: DataLoader is not an Optimizer


======================================================================  
🚀 Training model on DATASET_2 dataset
======================================================================  
___________🧩 Using 10% of training data___________  
Epoch 1/6 | Train Loss: 0.6624, Train Acc: 0.6061 | Val Loss: 0.5799, Val Acc: 0.7219                
Epoch 2/6 | Train Loss: 0.5179, Train Acc: 0.7594 | Val Loss: 0.5534, Val Acc: 0.7312                
Epoch 3/6 | Train Loss: 0.4341, Train Acc: 0.8096 | Val Loss: 0.4320, Val Acc: 0.8120                
Epoch 4/6 | Train Loss: 0.3945, Train Acc: 0.8308 | Val Loss: 0.4003, Val Acc: 0.8252              
Epoch 5/6 | Train Loss: 0.3716, Train Acc: 0.8404 | Val Loss: 0.3567, Val Acc: 0.8496              
Epoch 6/6 | Train Loss: 0.3588, Train Acc: 0.8458 | Val Loss: 0.3478, Val Acc: 0.8523              
___________🧩 Using 50% of training data___________  
Epoch 1/6 | Train Loss: 0.5106, Train Acc: 0.7353 | Val Loss: 0.5287, Val Acc: 0.6781              
Epoch 2/6 | Train Loss: 0.3578, Train Acc: 0.8452 | Val Loss: 0.5693, Val Acc: 0.6529              
Epoch 3/6 | Train Loss: 0.3324, Train Acc: 0.8569 | Val Loss: 0.5073, Val Acc: 0.7133              
Epoch 4/6 | Train Loss: 0.3191, Train Acc: 0.8620 | Val Loss: 0.5989, Val Acc: 0.6296              
Epoch 5/6 | Train Loss: 0.3086, Train Acc: 0.8663 | Val Loss: 0.4869, Val Acc: 0.7649              
Epoch 6/6 | Train Loss: 0.3027, Train Acc: 0.8693 | Val Loss: 0.4661, Val Acc: 0.7953              
__________🧩 Using 100% of training data___________  
Epoch 1/6 | Train Loss: 0.4213, Train Acc: 0.8050 | Val Loss: 0.3473, Val Acc: 0.8572              
Epoch 2/6 | Train Loss: 0.3203, Train Acc: 0.8626 | Val Loss: 0.3535, Val Acc: 0.8393              
Epoch 3/6 | Train Loss: 0.3016, Train Acc: 0.8692 | Val Loss: 0.4240, Val Acc: 0.7920              
Epoch 4/6 | Train Loss: 0.2923, Train Acc: 0.8723 | Val Loss: 0.3619, Val Acc: 0.8312              
Epoch 5/6 | Train Loss: 0.2845, Train Acc: 0.8753 | Val Loss: 0.3901, Val Acc: 0.8187              
Epoch 6/6 | Train Loss: 0.2788, Train Acc: 0.8772 | Val Loss: 0.3979, Val Acc: 0.8223              

✅ All datasets trained successfully!  

======================================================================  
📈 Final Validation Accuracy Summary  
======================================================================  
dataset_2            | Val Acc: 0.8223 | Val Loss: 0.3979  

In [7]:
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import torch


In [8]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for i in range(0, len(test_url_tensor), 64):  # batch size 64
        batch_x = test_url_tensor[i:i+64].to(device)
        batch_y = test_labels_tensor[i:i+64].to(device)
        
        outputs = model(batch_x)
        preds = (outputs >= 0.5).long().squeeze(1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())


NameError: name 'test_url_tensor' is not defined

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN_BiLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim=128):
        super(CNN_BiLSTM, self).__init__()
        
        # Embedding layer
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim)
        
        # Conv blocks
        self.conv1 = nn.Conv1d(embed_dim, 256, kernel_size=8, padding=4)
        self.pool1 = nn.MaxPool1d(2)

        self.conv2 = nn.Conv1d(256, 128, kernel_size=5, padding=2)
        self.pool2 = nn.MaxPool1d(2)
        
        # BiLSTM
        self.lstm = nn.LSTM(input_size=128, hidden_size=64, num_layers=1,
                            batch_first=True, bidirectional=True)
        
        # Fully connected layers
        self.fc1 = nn.Linear(64*2, 128)  # 64*2 because bidirectional
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 1)  # binary output
        
    def forward(self, x):
        # x: (batch_size, seq_len)
        x = self.embedding(x)  # (batch_size, seq_len, embed_dim)
        x = x.permute(0, 2, 1)  # (batch_size, embed_dim, seq_len) for Conv1d
        
        # Conv block 1
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        
        # Conv block 2
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        
        # Prepare for LSTM: (batch_size, seq_len, features)
        x = x.permute(0, 2, 1)
        
        # BiLSTM
        lstm_out, _ = self.lstm(x)  # lstm_out: (batch_size, seq_len, 2*hidden_size)
        
        # Take the last timestep
        x = lstm_out[:, -1, :]  # (batch_size, 128)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.fc2(x)
        
        return x


In [ ]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ============================================================
# 🔧 Training Config
# ============================================================
num_epochs = 3
lr = 0.001
batch_print_interval = 1

# Dictionary to store all dataset results
all_results = {}

# ============================================================
# 🔁 Loop Over Each Dataset
# ============================================================
for dataset_name, loaders in dataloader_dict.items():
    print("\n" + "="*70)
    print(f"🚀 Training CNN-BiLSTM on {dataset_name.upper()} dataset")
    print("="*70)

    train_loader = loaders["train_loader"]
    val_loader = loaders["val_loader"]

    model = CNN_BiLSTM(vocab_size=len(vocab)).to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # Track metrics
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        correct_train = 0
        total_train = 0

        for batch_idx, (batch_x, batch_y) in enumerate(train_loader):
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device).float().unsqueeze(1)
            batch_y = torch.clamp(batch_y, 0, 1)

            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            preds = (torch.sigmoid(outputs) >= 0.5).float()
            acc = (preds == batch_y).float().mean().item()

            train_loss += loss.item() * batch_x.size(0)
            correct_train += (preds == batch_y).sum().item()
            total_train += batch_x.size(0)

            if (batch_idx + 1) % batch_print_interval == 0:
                sys.stdout.write(f"\r[{dataset_name}] Epoch {epoch+1}/{num_epochs} | "
                                 f"Batch {batch_idx+1}/{len(train_loader)} | "
                                 f"Loss: {loss.item():.4f}, Acc: {acc:.4f}")
                sys.stdout.flush()

        avg_train_loss = train_loss / total_train
        train_acc = correct_train / total_train
        print()  # newline

        # === Validation ===
        model.eval()
        val_loss = 0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x = batch_x.to(device)
                batch_y = batch_y.to(device).float().unsqueeze(1)
                batch_y = torch.clamp(batch_y, 0, 1)

                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                val_loss += loss.item() * batch_x.size(0)

                preds = (torch.sigmoid(outputs) >= 0.5).float()
                correct_val += (preds == batch_y).sum().item()
                total_val += batch_x.size(0)

        avg_val_loss = val_loss / total_val
        val_acc = correct_val / total_val

        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)

        print(f"[{dataset_name}] Epoch {epoch+1}/{num_epochs} | "
              f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f} | "
              f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}")

    # === Save results for this dataset ===
    all_results[dataset_name] = {
        "train_losses": train_losses,
        "val_losses": val_losses,
        "train_accs": train_accs,
        "val_accs": val_accs,
        "final_val_acc": val_accs[-1],
        "final_val_loss": val_losses[-1]
    }

print("\n✅ All datasets trained successfully!")

# ============================================================
# 📊 Summary of Final Results
# ============================================================
print("\n" + "="*70)
print("📈 Final Validation Summary (CNN-BiLSTM)")
print("="*70)
for name, res in all_results.items():
    print(f"{name:<20} | Val Acc: {res['final_val_acc']:.4f} | Val Loss: {res['final_val_loss']:.4f}")



🚀 Training CNN-BiLSTM on DATASET1 dataset
[dataset1] Epoch 1/3 | Batch 102/102 | Loss: 0.2697, Acc: 0.8942
[dataset1] Epoch 1/3 | Train Loss: 0.4063, Train Acc: 0.8384 | Val Loss: 0.2719, Val Acc: 0.8974
[dataset1] Epoch 2/3 | Batch 102/102 | Loss: 0.1222, Acc: 0.9658
[dataset1] Epoch 2/3 | Train Loss: 0.1782, Train Acc: 0.9378 | Val Loss: 0.1227, Val Acc: 0.9609
[dataset1] Epoch 3/3 | Batch 102/102 | Loss: 0.0869, Acc: 0.9730
[dataset1] Epoch 3/3 | Train Loss: 0.1033, Train Acc: 0.9677 | Val Loss: 0.0860, Val Acc: 0.9731

🚀 Training CNN-BiLSTM on DATASET2 dataset
[dataset2] Epoch 1/3 | Batch 47/47 | Loss: 0.0809, Acc: 0.9864
[dataset2] Epoch 1/3 | Train Loss: 0.3467, Train Acc: 0.8455 | Val Loss: 0.0146, Val Acc: 0.9977
[dataset2] Epoch 2/3 | Batch 47/47 | Loss: 0.0022, Acc: 1.0000
[dataset2] Epoch 2/3 | Train Loss: 0.0140, Train Acc: 0.9977 | Val Loss: 0.0102, Val Acc: 0.9981
[dataset2] Epoch 3/3 | Batch 47/47 | Loss: 0.0022, Acc: 1.0000
[dataset2] Epoch 3/3 | Train Loss: 0.0122, Tr